In [0]:
#from google.colab import files
import numpy as np
import tensorflow as tf
import time

In [2]:
'''This is for the Colab's notebook'''
#uploaded = files.upload()

Saving nietzsche.txt to nietzsche.txt


In [3]:
with open('nietzsche.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

print(encoded[:100])

[35 81 66 79 32 36 66 46 46 46 31 18 35 35 47 31  8 73 33 24 22 57 78 22
 24 50 67 71 22 57 24  0 14 24 78 24 77 10 27 78 62 44 44 77 57 78 22 24
 22 57 15 62 23 24  8 14 24 22 57 15 67 15 24 62 10 22 24  9 67 10 71 62
 45 46 75 10 67 24 14 71 14 21 15 34 22  0 62  9 24 22 57 78 22 24 78  2
  2 24 21 57]


In [0]:
def get_batches(arr, n_seqs, n_steps):

    # Get the batch size and number of batches we can make
    batch_size = n_seqs * n_steps 
    n_batches =  len(arr)//batch_size
    
    # Keep only enough characters to make full batches
    arr =  arr[:n_batches*batch_size]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs,-1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:,n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros(x.shape)
        y[:,:-1],y[:,-1] = x[:,1:] ,x[:,0]
        yield x, y

In [0]:
batch_size = 100        # Sequences per batch
num_steps = 150         # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001   # Learning rate
keep_prob = 0.5         # Dropout keep probability
epochs = 40
save_every_n = 200      # Save every N iterations

In [0]:
def build_inputs(batch_size, num_steps):
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name="inputs")
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name="targets")
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")

    return inputs, targets, keep_prob

def build_lstm(lstm_size, num_layers, batch_size, keep_prob):

    def build_cell(lstm_size, keep_prob):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop

    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)

    return cell, initial_state

def build_output(lstm_output, in_size, out_size):
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])

    with tf.variable_scope("softmax"):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size),stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))

    logits = tf.matmul(x,softmax_w) + softmax_b
    out = tf.nn.softmax(logits, name='predictions')
    return out, logits

def build_loss(logits, targets, lstm_size, num_classes):
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)

    return loss

def build_optimizer(loss, learning_rate, grad_clip):
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))

    return optimizer

class CharRNN:
    def __init__(self, num_classes, batch_size, num_steps, lstm_size,
                    num_layers, learning_rate,
                    grad_clip=5, sampling=False):
        
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps
        tf.reset_default_graph()

        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cells
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, keep_prob)

        ## Run the data through RNN layers
        # First one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)

        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state

        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)

        # Loss and Optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [0]:
def train(encoded, vocab):
    model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                    lstm_size=lstm_size, num_layers=num_layers,
                    learning_rate=learning_rate)
    
    saver = tf.train.Saver(max_to_keep=100)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # saver.restore(sess,"checkpoints/______.ckpt")
        counter = 0
        for e in range(epochs):
            new_state = sess.run(model.initial_state)
            loss = 0
            for x, y in get_batches(encoded, batch_size, num_steps):
                counter += 1
                start = time.time()
                feed = {model.inputs:x,
                        model.targets:y,
                        model.keep_prob:keep_prob,
                        model.initial_state:new_state
                        }
                batch_loss, new_state, _ = sess.run([model.loss,
                                                 model.final_state,
                                                 model.optimizer],
                                                 feed_dict=feed)
                end = time.time()
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))

                if (counter % save_every_n == 0):
                    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

        saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

In [0]:
def pick_top_n(preds, vocab_size, top_n=3):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

def sample(checkpoint, vocab, vocab_to_int, int_to_vocab, n_samples, lstm_size, vocab_size, seed="The "):
    samples = [c for c in seed]
    model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                    lstm_size=lstm_size, num_layers=num_layers,
                    learning_rate=learning_rate, sampling=True)
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in seed:
            x = np.zeros((1, 1))
            x[0, 0] = vocab_to_int[c]
            feed = {model.inputs:x,
                    model.keep_prob:1.0,
                    model.initial_state:new_state
                    }
            preds, new_state = sess.run([model.prediction, model.final_state],
                                        feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0, 0] = c
            feed = {model.inputs:x,
                    model.keep_prob:1.0,
                    model.initial_state:new_state
                    }
            preds, new_state = sess.run([model.prediction, model.final_state],
                                        feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])

    return "".join(samples)

In [35]:
train(encoded, vocab)

Epoch: 1/40...  Training Step: 1...  Training loss: 4.4329...  0.6840 sec/batch
Epoch: 1/40...  Training Step: 2...  Training loss: 4.3549...  0.5096 sec/batch
Epoch: 1/40...  Training Step: 3...  Training loss: 3.9355...  0.5030 sec/batch
Epoch: 1/40...  Training Step: 4...  Training loss: 5.7739...  0.5170 sec/batch
Epoch: 1/40...  Training Step: 5...  Training loss: 4.3575...  0.5108 sec/batch
Epoch: 1/40...  Training Step: 6...  Training loss: 3.7824...  0.5223 sec/batch
Epoch: 1/40...  Training Step: 7...  Training loss: 3.6650...  0.5045 sec/batch
Epoch: 1/40...  Training Step: 8...  Training loss: 3.5711...  0.5055 sec/batch
Epoch: 1/40...  Training Step: 9...  Training loss: 3.5029...  0.5109 sec/batch
Epoch: 1/40...  Training Step: 10...  Training loss: 3.4793...  0.5175 sec/batch
Epoch: 1/40...  Training Step: 11...  Training loss: 3.4260...  0.5234 sec/batch
Epoch: 1/40...  Training Step: 12...  Training loss: 3.4061...  0.5135 sec/batch
Epoch: 1/40...  Training Step: 13... 

Epoch: 4/40...  Training Step: 147...  Training loss: 2.8906...  0.5196 sec/batch
Epoch: 4/40...  Training Step: 148...  Training loss: 2.8127...  0.5219 sec/batch
Epoch: 4/40...  Training Step: 149...  Training loss: 2.8384...  0.5201 sec/batch
Epoch: 4/40...  Training Step: 150...  Training loss: 2.8046...  0.5020 sec/batch
Epoch: 4/40...  Training Step: 151...  Training loss: 2.7773...  0.5133 sec/batch
Epoch: 4/40...  Training Step: 152...  Training loss: 2.7845...  0.5125 sec/batch
Epoch: 4/40...  Training Step: 153...  Training loss: 2.8048...  0.5186 sec/batch
Epoch: 4/40...  Training Step: 154...  Training loss: 2.8010...  0.5213 sec/batch
Epoch: 4/40...  Training Step: 155...  Training loss: 2.7561...  0.5172 sec/batch
Epoch: 4/40...  Training Step: 156...  Training loss: 2.7707...  0.5072 sec/batch
Epoch: 4/40...  Training Step: 157...  Training loss: 2.7768...  0.5178 sec/batch
Epoch: 4/40...  Training Step: 158...  Training loss: 2.7691...  0.5189 sec/batch
Epoch: 4/40...  

Epoch: 8/40...  Training Step: 291...  Training loss: 2.3638...  0.5136 sec/batch
Epoch: 8/40...  Training Step: 292...  Training loss: 2.3578...  0.5151 sec/batch
Epoch: 8/40...  Training Step: 293...  Training loss: 2.3565...  0.5168 sec/batch
Epoch: 8/40...  Training Step: 294...  Training loss: 2.3663...  0.5160 sec/batch
Epoch: 8/40...  Training Step: 295...  Training loss: 2.3750...  0.5288 sec/batch
Epoch: 8/40...  Training Step: 296...  Training loss: 2.3547...  0.5243 sec/batch
Epoch: 8/40...  Training Step: 297...  Training loss: 2.3377...  0.5318 sec/batch
Epoch: 8/40...  Training Step: 298...  Training loss: 2.3315...  0.5071 sec/batch
Epoch: 8/40...  Training Step: 299...  Training loss: 2.3521...  0.5226 sec/batch
Epoch: 8/40...  Training Step: 300...  Training loss: 2.3488...  0.5193 sec/batch
Epoch: 8/40...  Training Step: 301...  Training loss: 2.3607...  0.5172 sec/batch
Epoch: 8/40...  Training Step: 302...  Training loss: 2.3607...  0.5224 sec/batch
Epoch: 8/40...  

Epoch: 11/40...  Training Step: 435...  Training loss: 2.1360...  0.5152 sec/batch
Epoch: 11/40...  Training Step: 436...  Training loss: 2.1371...  0.5159 sec/batch
Epoch: 11/40...  Training Step: 437...  Training loss: 2.1302...  0.5244 sec/batch
Epoch: 11/40...  Training Step: 438...  Training loss: 2.1702...  0.5108 sec/batch
Epoch: 11/40...  Training Step: 439...  Training loss: 2.1516...  0.5061 sec/batch
Epoch: 11/40...  Training Step: 440...  Training loss: 2.1354...  0.5236 sec/batch
Epoch: 12/40...  Training Step: 441...  Training loss: 2.1720...  0.5229 sec/batch
Epoch: 12/40...  Training Step: 442...  Training loss: 2.1251...  0.5200 sec/batch
Epoch: 12/40...  Training Step: 443...  Training loss: 2.1293...  0.5286 sec/batch
Epoch: 12/40...  Training Step: 444...  Training loss: 2.1181...  0.5066 sec/batch
Epoch: 12/40...  Training Step: 445...  Training loss: 2.1527...  0.5154 sec/batch
Epoch: 12/40...  Training Step: 446...  Training loss: 2.1130...  0.5196 sec/batch
Epoc

Epoch: 15/40...  Training Step: 579...  Training loss: 1.9775...  0.5137 sec/batch
Epoch: 15/40...  Training Step: 580...  Training loss: 1.9869...  0.5205 sec/batch
Epoch: 15/40...  Training Step: 581...  Training loss: 1.9834...  0.5200 sec/batch
Epoch: 15/40...  Training Step: 582...  Training loss: 1.9962...  0.5203 sec/batch
Epoch: 15/40...  Training Step: 583...  Training loss: 1.9870...  0.5232 sec/batch
Epoch: 15/40...  Training Step: 584...  Training loss: 1.9694...  0.5088 sec/batch
Epoch: 15/40...  Training Step: 585...  Training loss: 1.9449...  0.5199 sec/batch
Epoch: 15/40...  Training Step: 586...  Training loss: 1.9711...  0.5059 sec/batch
Epoch: 15/40...  Training Step: 587...  Training loss: 1.9702...  0.5179 sec/batch
Epoch: 15/40...  Training Step: 588...  Training loss: 1.9655...  0.5158 sec/batch
Epoch: 15/40...  Training Step: 589...  Training loss: 1.9736...  0.5039 sec/batch
Epoch: 15/40...  Training Step: 590...  Training loss: 1.9719...  0.5141 sec/batch
Epoc

...

Epoch: 26/40...  Training Step: 1011...  Training loss: 1.6930...  0.5120 sec/batch
Epoch: 26/40...  Training Step: 1012...  Training loss: 1.6761...  0.5272 sec/batch
Epoch: 26/40...  Training Step: 1013...  Training loss: 1.6569...  0.5161 sec/batch
Epoch: 26/40...  Training Step: 1014...  Training loss: 1.6824...  0.5066 sec/batch
Epoch: 26/40...  Training Step: 1015...  Training loss: 1.7050...  0.5045 sec/batch
Epoch: 26/40...  Training Step: 1016...  Training loss: 1.6915...  0.5264 sec/batch
Epoch: 26/40...  Training Step: 1017...  Training loss: 1.6414...  0.5113 sec/batch
Epoch: 26/40...  Training Step: 1018...  Training loss: 1.6586...  0.5274 sec/batch
Epoch: 26/40...  Training Step: 1019...  Training loss: 1.6629...  0.5262 sec/batch
Epoch: 26/40...  Training Step: 1020...  Training loss: 1.6618...  0.5227 sec/batch
Epoch: 26/40...  Training Step: 1021...  Training loss: 1.6572...  0.5186 sec/batch
Epoch: 26/40...  Training Step: 1022...  Training loss: 1.6715...  0.5227 se

Epoch: 29/40...  Training Step: 1155...  Training loss: 1.5440...  0.5157 sec/batch
Epoch: 29/40...  Training Step: 1156...  Training loss: 1.5911...  0.5141 sec/batch
Epoch: 29/40...  Training Step: 1157...  Training loss: 1.5678...  0.5207 sec/batch
Epoch: 29/40...  Training Step: 1158...  Training loss: 1.6271...  0.5113 sec/batch
Epoch: 29/40...  Training Step: 1159...  Training loss: 1.5963...  0.5177 sec/batch
Epoch: 29/40...  Training Step: 1160...  Training loss: 1.5603...  0.5230 sec/batch
Epoch: 30/40...  Training Step: 1161...  Training loss: 1.6321...  0.5121 sec/batch
Epoch: 30/40...  Training Step: 1162...  Training loss: 1.5823...  0.5233 sec/batch
Epoch: 30/40...  Training Step: 1163...  Training loss: 1.5844...  0.5121 sec/batch
Epoch: 30/40...  Training Step: 1164...  Training loss: 1.5849...  0.5135 sec/batch
Epoch: 30/40...  Training Step: 1165...  Training loss: 1.5986...  0.5235 sec/batch
Epoch: 30/40...  Training Step: 1166...  Training loss: 1.5836...  0.5137 se

Epoch: 33/40...  Training Step: 1299...  Training loss: 1.5017...  0.5117 sec/batch
Epoch: 33/40...  Training Step: 1300...  Training loss: 1.5149...  0.5199 sec/batch
Epoch: 33/40...  Training Step: 1301...  Training loss: 1.5235...  0.5206 sec/batch
Epoch: 33/40...  Training Step: 1302...  Training loss: 1.5333...  0.5145 sec/batch
Epoch: 33/40...  Training Step: 1303...  Training loss: 1.5142...  0.5055 sec/batch
Epoch: 33/40...  Training Step: 1304...  Training loss: 1.5138...  0.5230 sec/batch
Epoch: 33/40...  Training Step: 1305...  Training loss: 1.4775...  0.5094 sec/batch
Epoch: 33/40...  Training Step: 1306...  Training loss: 1.5145...  0.5248 sec/batch
Epoch: 33/40...  Training Step: 1307...  Training loss: 1.4955...  0.5162 sec/batch
Epoch: 33/40...  Training Step: 1308...  Training loss: 1.5073...  0.5168 sec/batch
Epoch: 33/40...  Training Step: 1309...  Training loss: 1.5154...  0.5206 sec/batch
Epoch: 33/40...  Training Step: 1310...  Training loss: 1.5190...  0.5216 se

Epoch: 37/40...  Training Step: 1443...  Training loss: 1.4928...  0.5164 sec/batch
Epoch: 37/40...  Training Step: 1444...  Training loss: 1.4772...  0.5129 sec/batch
Epoch: 37/40...  Training Step: 1445...  Training loss: 1.4765...  0.5202 sec/batch
Epoch: 37/40...  Training Step: 1446...  Training loss: 1.4683...  0.5140 sec/batch
Epoch: 37/40...  Training Step: 1447...  Training loss: 1.4483...  0.5072 sec/batch
Epoch: 37/40...  Training Step: 1448...  Training loss: 1.4552...  0.5231 sec/batch
Epoch: 37/40...  Training Step: 1449...  Training loss: 1.4572...  0.5239 sec/batch
Epoch: 37/40...  Training Step: 1450...  Training loss: 1.4607...  0.5153 sec/batch
Epoch: 37/40...  Training Step: 1451...  Training loss: 1.4915...  0.5121 sec/batch
Epoch: 37/40...  Training Step: 1452...  Training loss: 1.4781...  0.5121 sec/batch
Epoch: 37/40...  Training Step: 1453...  Training loss: 1.4622...  0.5138 sec/batch
Epoch: 37/40...  Training Step: 1454...  Training loss: 1.4852...  0.5214 se

Epoch: 40/40...  Training Step: 1587...  Training loss: 1.4046...  0.5208 sec/batch
Epoch: 40/40...  Training Step: 1588...  Training loss: 1.4209...  0.5262 sec/batch
Epoch: 40/40...  Training Step: 1589...  Training loss: 1.4331...  0.5166 sec/batch
Epoch: 40/40...  Training Step: 1590...  Training loss: 1.4327...  0.5114 sec/batch
Epoch: 40/40...  Training Step: 1591...  Training loss: 1.4288...  0.5185 sec/batch
Epoch: 40/40...  Training Step: 1592...  Training loss: 1.4311...  0.5094 sec/batch
Epoch: 40/40...  Training Step: 1593...  Training loss: 1.4442...  0.5072 sec/batch
Epoch: 40/40...  Training Step: 1594...  Training loss: 1.4552...  0.5202 sec/batch
Epoch: 40/40...  Training Step: 1595...  Training loss: 1.4035...  0.5247 sec/batch
Epoch: 40/40...  Training Step: 1596...  Training loss: 1.4494...  0.5230 sec/batch
Epoch: 40/40...  Training Step: 1597...  Training loss: 1.4226...  0.5237 sec/batch
Epoch: 40/40...  Training Step: 1598...  Training loss: 1.4754...  0.5211 se

In [0]:
files.download(tf.train.latest_checkpoint('checkpoints') + '.meta')
files.download(tf.train.latest_checkpoint('checkpoints') + '.index')
files.download(tf.train.latest_checkpoint('checkpoints') + '.data-00000-of-00001')

In [55]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, vocab, vocab_to_int, int_to_vocab, 1000, lstm_size, len(vocab), seed="King ")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1600_l512.ckpt
King there oneself alone
when the standard and metaphysics. They have always the
stated of the can to see in the present morality which they will
be a despection and the subjution of the concepsion, an indiversally
individual thers and instracted in the sullimination of the world with
a philosopher and the present their power. All their sours or also
there are self-decousive than to the master, and alse themselve
with he must both among teder our probect of heresing and subject of
their action and such a closy strong explanation, in order to the past
of construttry at a southern tratsed of a had to all thinh and traunds of
the soul andievy as and the constitutious, the strungefle still same,
the succersful instinct of the marbelian, which it is the some
ordinary think in a conception. The popenting tenses of things
are soul. Their it was the powerfol, and always the preyers of has
to bad consition on an extent of all